<font face='Times New Roman' size=4>
<span style="line-height: 0.5;">

## Numerical Analysis Final Project
#### Using Cholesky decompesition to solve linear equations systems
##### Matrix rank 1000
---
林之然(B11209005)

蔡知諺(B11209014)

<font face='Times New Roman' size=4>
<span style="line-height: 1.25;">

### Instruction to this Jupyter Notebook:
1. Assign the kernel where you normally run Python.  

2. Click the [<font face="標楷體">全部執行</font>] button or Cell > Run All to execute the entire code. 

3. Look through the codes and discussions.  

In [7]:
import numpy as np
import NAFunc as cheers

##### Generate random postive definite matrix and array in different size

In [8]:
# Generate a larger positive definite matrix
larger_size = 30
LA10 = np.random.rand(larger_size, larger_size)
LA10 = np.dot(LA10, LA10.T)

# Extract a submatrix with the desired rank
desired_rank = 10
A10 = LA10[:desired_rank, :desired_rank]

# Check if the submatrix is positive definite
eigenvalues = np.linalg.eigvals(A10)
is_positive_definite = np.all(eigenvalues > 0)

print("Submatrix with Rank", desired_rank)
print("Is positive definite:", is_positive_definite)
AX10 = np.random.random(desired_rank)

Submatrix with Rank 10
Is positive definite: True


In [9]:
# Generate a larger positive definite matrix
larger_size = 300
LA100 = np.random.rand(larger_size, larger_size)
LA100 = np.dot(LA100, LA100.T)

# Extract a submatrix with the desired rank
desired_rank = 100
A100 = LA100[:desired_rank, :desired_rank]

# Check if the submatrix is positive definite
eigenvalues = np.linalg.eigvals(A100)
is_positive_definite = np.all(eigenvalues > 0)

print("Submatrix with Rank", desired_rank)
print("Is positive definite:", is_positive_definite)
AX100 = np.random.random(desired_rank)

Submatrix with Rank 100
Is positive definite: True


In [10]:
# Generate a larger positive definite matrix
larger_size = 3000
LA1000 = np.random.rand(larger_size, larger_size)
LA1000 = np.dot(LA1000, LA1000.T)

# Extract a submatrix with the desired rank
desired_rank = 1000
A1000 = LA1000[:desired_rank, :desired_rank]

# Check if the submatrix is positive definite
eigenvalues = np.linalg.eigvals(A1000)
is_positive_definite = np.all(eigenvalues > 0)

print("Submatrix with Rank", desired_rank)
print("Is positive definite:", is_positive_definite)
AX1000 = np.random.random(desired_rank)

Submatrix with Rank 1000
Is positive definite: True


##### Define Cholesky decomposition algorism and other necessary function

In [11]:
def Cholesky_Decomposition(matrix):
    n = len(matrix)
    L = [[0.0] * n for _ in range(n)]

    for i in range(n):
        for j in range(i + 1):
            if i == j:
                L[i][j] = (matrix[i][i] - sum(L[i][k] ** 2 for k in range(j))) ** 0.5
            else:
                L[i][j] = (matrix[i][j] - sum(L[i][k] * L[j][k] for k in range(j))) / L[j][j]

    return L

In [12]:
def Transpose(A):
    return [[row[i] for row in A] for i in range(len(A[0]))]


def Inverse_Cholesky(L):
    """
    Calculate the inverse of a lower triangular matrix L from Cholesky decomposition.

    Args:
        L (list of lists): The lower triangular matrix L from Cholesky decomposition.

    Returns:
        list of lists: The inverse matrix of L.
    """
    n = len(L)
    inverse = [[0.0] * n for _ in range(n)]

    for i in range(n):
        inverse[i][i] = 1.0 / L[i][i]

        for j in range(i - 1, -1, -1):
            s = 0.0
            for k in range(j + 1, i + 1):
                s += L[i][k] * inverse[k][j]
            inverse[i][j] = -s / L[j][j]

    return inverse


def Backward_Substitution(U, b):
    """
    Perform backward substitution to solve a system of equations with an upper triangular matrix.

    Args:
        U (list of lists): The upper triangular matrix.
        b (list): The column vector representing the right-hand side of the equations.

    Returns:
        list: The solution vector x.
    """
    n = len(U)
    x = [0.0] * n

    for i in range(n - 1, -1, -1):
        x[i] = b[i] / U[i][i]
        for j in range(i - 1, -1, -1):
            b[j] -= U[j][i] * x[i]

    return x


def Lower_Triangular_Matrix_Vector_Multiply(matrix, vector):
    """
    Perform matrix-vector multiplication for a lower triangular matrix.

    Args:
        matrix (list of lists): The lower triangular matrix.
        vector (list): The input vector.

    Returns:
        list: The resulting vector.
    """
    # Get the dimensions of the matrix
    num_rows = len(matrix)
    num_cols = len(matrix[0])

    # Check if the dimensions match for multiplication
    if num_cols != len(vector):
        raise ValueError("Matrix and vector dimensions do not match for multiplication")

    # Initialize the resulting vector with zeros
    result = [0.0] * num_rows

    # Perform matrix-vector multiplication
    for i in range(num_rows):
        for j in range(i + 1):
            result[i] += matrix[i][j] * vector[j]

    return result

In [13]:
def Gaussian_Elimination(A, b):
    n = len(A)
    for i in range(n):

        # Pivoting
        max_row = i
        for j in range(i + 1, n):
            if abs(A[j][i]) > abs(A[max_row][i]):
                max_row = j
        A[i], A[max_row] = A[max_row], A[i]
        b[i], b[max_row] = b[max_row], b[i]

        # Forward
        for j in range(i + 1, n):
            factor = A[j][i] / A[i][i]
            b[j] -= factor * b[i]
            for k in range(i, n):
                A[j][k] -= factor * A[i][k]

    # Backward Sub
    x = [0.0] * n
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - sum(A[i][j] * x[j] for j in range(i + 1, n))) / A[i][i]

    return x

##### Independently Run the Cell to Estimate Operation Time

In [14]:
%%timeit  
# Estimate time: CPU times: 40-60 µs

L10 = Cholesky_Decomposition(A10)
X10C = Backward_Substitution(Transpose(L10),Lower_Triangular_Matrix_Vector_Multiply(Inverse_Cholesky(L10),AX10))

131 µs ± 708 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [15]:
%%timeit  
# Estimate time: CPU times: 120-160 µs

X10G = Gaussian_Elimination(A10, AX10)

232 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [16]:
%%timeit  
# Estimate time: CPU times: 40-60 ms

L100 = Cholesky_Decomposition(A100)
X100C = Backward_Substitution(Transpose(L100),Lower_Triangular_Matrix_Vector_Multiply(Inverse_Cholesky(L100),AX100))

52.1 ms ± 237 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit  
# Estimate time: CPU times: 120-160 ms

X100G = Gaussian_Elimination(A100, AX100)

153 ms ± 518 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%time  
# Estimate time: CPU times: 40-60 s

L1000 = Cholesky_Decomposition(A1000)
#X1C = cheers.BackwardSub(Transpose(L1),np.dot(inverse_cholesky(L1),AX1))
X1000C = Backward_Substitution(Transpose(L1000),Lower_Triangular_Matrix_Vector_Multiply(Inverse_Cholesky(L1000),AX1000))

CPU times: user 48.6 s, sys: 256 ms, total: 48.8 s
Wall time: 48.9 s


In [19]:
%%time  
# Estimate time: CPU times: 120-160 s

X1000G = Gaussian_Elimination(A1000, AX1000)

CPU times: user 2min 28s, sys: 582 ms, total: 2min 29s
Wall time: 2min 29s
